# Titanic ensemble modeling classifier

30/7/2021
* [**1. Introduction**](#1)
  * [1.1 Import library](#1.1)
  * [1.2 Loading data](#1.2)
* [**2. Getting familiar with data (Data analysis)**](#2) 
  * [2.1 Missing value](#2.1)
     * [2.1.1 Age](#2.1.1)
     * [2.1.2 Embarked](#2.1.2)
     * [2.1.3 Fare](#2.1.3)
     * [2.1.4 Cabin](#2.1.4)
  * [2.2 Target column distribution](#2.2)
  * [2.3 Correlations](#2.3)
* [**3. Feature engineer**](#3)
  * [3.1 Binning Coutinuous features](#3.1)
     * [3.1.1 Fare](#3.1.1)
     * [3.1.2 Age](#3.1.2)
  * [3.2 Frequency Encoded Binning](#3.2)
     * [3.2.1 Family size](#3.2.1)
     * [3.2.2 Ticket frequency](#3.2.2)
  * [3.3 Title & is Married](#3.3)
  * [3.4 Feature transformation](#3.4)
     * [3.4.1 Label encoding non-numerical features](#3.4.1)
     * [3.4.2 One-hot encoding the categorical features](#3.4.2)
* [**4. Modeling**](#4)
  * [4.1 Cross validation simple models](#4.1)
  * [4.2 Hyper parameters tunning](#4.2)
  * [4.3 Confusion matrix](#4.3)
  * [4.4 Ensemble model](#4.4)
  * [4.5 Submission](#4.5)

<a name='1'></a>
# 1. Introduction
This project is on the titanic which was on 14 April 1912. Before it was colliding with an ice berg and ended up at the bottom of the Alantic sea. She was the biggest moveable object at the time. She was call "unsinkable". Which was not true. Over a thousand people died in the icy sea that night

<img src="https://media.istockphoto.com/photos/titanic-and-iceberg-picture-id503132519?k=6&m=503132519&s=612x612&w=0&h=bipCwlzdAUIzTdYdZ6g-CyYWaKJ988T85veDl_3l1Gk=" style="width:600px;height:400px;">

*By the end of this notebook, we'll go through*
* **Data analysis** 
    * (Missing value, data visualization, correlation table, categorical features)
* **Feature engineering** 
    * (Binning continuous features, Frequency encoded binning, Feature transformation)
* **Modeling** 
    * (Cross validation technique, Hyper parameter tunning, Confusion matrix)

<a name='1.1'></a>
## 1.1 Import library

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
SEED = 42

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

import warnings
import time
warnings.filterwarnings('ignore')

<a name='1.2'></a>
## 1.2 Loading data
- Training set has 891 rows, testing set has 418 rows
- There are 12 features in training set and 11 features in testing set
- One extra feature in training set is "Survived" which is the target feature

In [ ]:
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

def divide_df(all_data):
    # Returns divided dfs of training and test set
    return all_data.loc[:890], all_data.loc[891:].drop(['Survived'], axis=1)

df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')
df_all = concat_df(df_train, df_test)

df_train.name = 'Training Set'
df_test.name = 'Test Set'
df_all.name = 'All Set' 

dfs = [df_train, df_test]

print(f'Number of Training Examples = {df_train.shape[0]}')
print(f'Number of Test Examples = {df_test.shape[0]}\n')
print(f'Training X Shape = {df_train.shape}')
print(f'Training y Shape = {df_train["Survived"].shape[0]}\n')
print(f'Test X Shape = {df_test.shape}')
print(f'Test y Shape = {df_test.shape[0]}\n')
print(df_train.columns)
print(df_test.columns)

<a name='2'></a>
# 2. Getting familiar with data
- **PassengerId** is the unique id of the row and it doesn't have any effect on target
- **Survived** is the target variable we are trying to predict (*0=Not Survived* or *1=Survived*)
- **Pclass** (Passenger class) is the socio-economic status of the passenger and it is a categorical ordinal feature which has 3 unique values (*1=Upper Class*, *2=Middle Class* or *3=Lower Class*)
- **Name**, **Sex** and **Age** are self-explantory
- **SipSp** the total number of passenger's siblings and spouses
- **Parch** the total number of passenger's parent and children
- **Ticket** the ticket number of passenger
- **Fare** Passenger's fare
- **Cabin** is the cabin number of passenger
- **Embarked** is port of embarkation and it is a categorical feature which has 3 unique values (*C=Cherbourg*, *Q=Queenstown* or *S=Southampton*)

In [ ]:
print(df_train.info())
df_train.head(3)

<a name='2.1'></a>
## 2.1 Missing value & Visualization
(Which columns contain missing value?)
<br>As we seen below, the *display_missing()* function count number of missing value in each feature of both training and testing dataset
- Columns in training set contained missing value: **Age**, **Cabin**, **Embarked**
- Columns in testing set contained missing value: **Age**, **Fare**, **Cabin**
<br>

**Remark**
- It's more convinient to concatunate training and testing set when dealing with features containing missing value.
- The missing value of "Age", "Embarked" and "fare" is smaller compared to total samples, but roughly 80% of "Cabin" value is missing
- The missing value of "Age", "Embarked" and "fare" can be filled with descriptive statistic (median or mean) but this shouldn't be applied for "Cabin" feature

In [ ]:
msno.matrix(df_train)
plt.show()

In [ ]:
for df in dfs:
    print(f'Missing values in {df.name}')
    print(df.isnull().sum(), "\n")

<a name='2.1.1'></a>
### 2.1.1 Age
- Use correlation table to find the most appropriate features to groupped by for filling the missing values

In [ ]:
# Correlation table between features
df_all_corr = df_all.corr().abs().unstack().sort_values(kind="mergesort", ascending=False).reset_index()
df_all_corr.rename(columns={0:'Correlation'}, inplace=True)
df_all_corr.loc[df_all_corr['level_0'] == 'Age']

- **Sex** level can be considered as the second level of groupby while filling the missing value of **Age**
- By looking at the table, the higher the level of Pclass, the higher the median of Age's values & the Age's value of males tends to be higher than those of females

In [ ]:
# Fill Age's null val by the median grouped by "Sex" and "Pclass" features
df_all['Age'] = df_all.groupby(['Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))

In [ ]:
train, test = divide_df(df_all)

g = sns.FacetGrid(train, hue="Survived", aspect=2, height=5)
g.map(sns.kdeplot, "Age", alpha=.7, shade=True).add_legend()
plt.show()

<a name='2.1.2'></a>
### 2.1.2 Embarked

In [ ]:
# Missing value in "Embarked" column
df_all.loc[df_all['Embarked'].isnull()]

As we see, both passengers are **female**, with **Pclass 1** & paid **\$80** for a fare. Let's see how fare is distributed Embarked and Pclass features

In [ ]:
train, test = divide_df(df_all)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 10))
sns.boxplot(x='Embarked', y='Fare', hue='Pclass', data=train, ax=axs[0])
sns.boxplot(x='Embarked', y='Fare', hue='Pclass', data=test, ax=axs[1])
plt.show()

Here, 1st class is blue box. The mean of money paid for a fare, which is closed to \$80, belong to Embarked C in both training and testing dataset --> The missing piece is "C"

In [ ]:
# Filling Embarked's missing value with S
df_all['Embarked'] = df_all['Embarked'].fillna("C")

<a name='2.1.3'></a>
### 2.1.3 Fare
- Fare is the cost of ticket. There is only 1 missing value in the dataset

--> Just use median of Fare to fill in the missing piece

In [ ]:
# Missing value in "Fare" column
df_all.loc[df_all['Fare'].isnull()]

In [ ]:
med_fare = df_all.Fare.median()
df_all['Fare'] = df_all['Fare'].fillna(med_fare)

In [ ]:
df_train, df_test = divide_df(df_all)

plt.figure(figsize=(12, 8))
g = sns.FacetGrid(df_train, hue="Survived", aspect=2, height=5)
g.map(sns.kdeplot, 'Fare', shade= True).add_legend()
g.set(xlim=(-100, df_train['Fare'].max()))

plt.show()

<a name='2.1.4'></a>
### 2.1.4 Cabin
- The large portion (nearly 80%) of cabin feature is missing and the feature itself can not be ignored completely because cabin is essential in deciding the survival rate of customers.

--> Therefore, It'd be appropriate to separate all the null cabin values as a cabin "M"

- The Cabin value looks like C123, it turns out the 1st letter represents the deck in which cabin is located.

--> We need to separate the column "Cabin" into the "Deck" column only and the remaining number are redundant (should be removed)

In [ ]:
nan_cabin_percent = round(df_all['Cabin'].isnull().mean()*100, 2)
print(f"Proportion of missing values in \"Cabin\" feature is {nan_cabin_percent}%")

In [ ]:
# Create col "Deck" contained only 1st letter of column "Cabin" & assign "M" to null
df_all['Deck'] = df_all['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'X')

# Transform passenger with deck T to A
idx = df_all.loc[df_all['Deck'] == "T"].index
df_all.loc[idx, 'Deck'] = 'A'

df_all['Deck'].value_counts()

In [ ]:
train, test = divide_df(df_all)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
sns.countplot(train['Deck'], ax=ax1)
ax1.set_title('Count passengers each deck (Train data only)', fontsize=15)

sns.countplot(x='Deck', hue='Survived', data=train, ax=ax2, palette='rocket')
ax2.set_title('Relationship between Survival and Deck (Train data only)', fontsize=15)
plt.show()

The survival rate is just one-half of the mortality rate in Deck "X". It logically make sense, because deck "X" represent the missing infomations & it's hard to derive those values back from the non-survived passengers.

### 2.1.5 Deck & Survived
'Decks' is now a high-cardinality feature -> We'll group decks value based on their similarity
- A, B and C decks are labeled as ABC because all of them have only 1st class passengers
- D and E decks are labeled as DE because both of them have similar passenger class distribution and same survival rate
- F and G decks are labeled as FG because of the same reason above
- M deck doesn't need to be grouped with other decks because it is very different from others and has the lowest survival rate.

(NFM) "High-cardinality" feature is the feature which is unique and uncommon (too many unique values)

In [ ]:
df_all['Deck'] = df_all['Deck'].replace(['A', 'B', 'C'], 'ABC')
df_all['Deck'] = df_all['Deck'].replace(['D', 'E'], 'DE')
df_all['Deck'] = df_all['Deck'].replace(['F', 'G'], 'FG')

df_all['Deck'].value_counts()

In [ ]:
train, test = divide_df(df_all)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
sns.countplot(train['Deck'], palette='Set2', ax=ax1)
ax1.set_title('Count passengers each deck (Train data only)', fontsize=15)

sns.countplot(x='Deck', hue='Survived', data=train, ax=ax2, palette='hls')
ax2.set_title('Relationship between Survival and Deck (Train data only)', fontsize=15)
plt.show()

### 2.1.6 Pclass & Survived

In [ ]:
train, test = divide_df(df_all)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 7))
sns.countplot(train['Pclass'], palette='Set2', ax=ax1)
ax1.set_title("Count passengers in each Pclass")

sns.barplot(x='Sex', y='Survived', hue='Pclass', data=train, ax=ax2, palette='Set1')
ax2.set_title("Survived distributed among Sex and Pclass")

sns.factorplot('Pclass', 'Survived', order=[1, 2, 3], data=train)

for i in range(1, 4):
    temp = train.loc[df_train['Pclass']==i]['Survived'].value_counts()
    print(f'Percentage of Pclass {i} survived over total Pclass {i}: {round(temp[1]/sum(temp), 2)*100}%')


- The majority of passengers belongs to Pclass 3 which accounts for nearly 55%.
- By following the plot, the relationship between Pclass and Sex is essential for the survival prediction. Pclass 1 and 2 are almost guaranteed survival for women, while Pclass 2 and 3 are extremely bad for men to survived.
- The survival percentage of Pclass 1 is around 67% compared to 47% in 2nd class & 24% in 3rd class

In [ ]:
# Drop feature Cabin
df_all.drop('Cabin', axis=1, inplace=True)

# Separate training and testing data
df_train, df_test = divide_df(df_all)

print('Missing values in Training dataset')
print(df_train.isnull().sum(), "\n")
print('\n Missing values in Testing dataset')
print(df_test.isnull().sum(), "\n")

<a name='2.2'></a>
## 2.2 Target column distribution

In [ ]:
# Compute survival percentage in training dataset
not_survived = df_train['Survived'].loc[df_train['Survived']==0].count()
survived = df_train['Survived'].loc[df_train['Survived']==1].count()
print(f'{not_survived} of {not_survived+survived} passengers survived and it is the 38.38% of the training set.')
print(f'{survived} of {not_survived+survived} passengers survived and it is the 61.62% of the training set.')

In [ ]:
# Visualize number of survival and non_survivals in training dataset
survived_rate = [survived, not_survived]
bar_count = ['Survived', 'Not survived']

fig, axs = plt.subplots(1, 2, figsize=(9, 7))
sns.countplot(df_train['Survived'], facecolor=(0, 0, 0, 0),
              linewidth=5, edgecolor=sns.color_palette("dark", 3), ax=axs[0])
sns.countplot(x='Sex', hue='Survived', data=df_train, ax=axs[1], palette='husl')

axs[0].set_ylabel('Count', size=15, labelpad=5)
axs[1].set_ylabel('Count', size=15, labelpad=5)
axs[0].set_xlabel('Survived', size=15, labelpad=5)
axs[1].set_xlabel('Sex', size=15, labelpad=5)

axs[0].set_title('Training Set Survival Distribution', size=13, y=1.05)
axs[1].set_title('Survival distributed among sex', size=13, y=1.05)
plt.show()

- Of 1309 people on the titanic, the majority of 64.4% are male. Within the 1st plot, nearly 550 passengers died compared for only 350 people survived. In the 2nd plot with training data, 81.1% men and 25.8% of women died. Depending on that, "Sex" plays an important role in predicting the survival

<a name='2.3'></a>
## 2.3 Correlations
- Correlation is a statistical term which in simple explanation refers to how close two variables having linear relationship with each other

For example, 2 variables are linearly dependent (x and y are dependent on each other as x=2y). While 2 variable are independent (# of parents & Pclass are not close to each other)

*(Quest ask myself!) Why negative values exist in correlation table?*
- It means that there is an inverse relationship between 2 variables tested
- For example, the snowfall-driver is negative correlation b.c the increase the amount of snowfall, the few the number of driver on the street

In [ ]:
# Construct correlation table for Training data
df_train_corr = df_train.drop(['PassengerId'], axis=1).corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_train_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
# Remove odd row's index b.c they're replicated by the even row's index
df_train_corr.drop(df_train_corr.iloc[1::2].index, inplace=True)
# Drop row having correlation value 1.0
df_train_corr_nd = df_train_corr.drop(df_train_corr[df_train_corr['Correlation Coefficient'] == 1.0].index)

# Construct correlation table for Testing data (Steps same as training data)
df_test_corr = df_test.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_test_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_test_corr.drop(df_test_corr.iloc[1::2].index, inplace=True)
df_test_corr_nd = df_test_corr.drop(df_test_corr[df_test_corr['Correlation Coefficient'] == 1.0].index)

In [ ]:
# Visualize correlation table by heatmap
fig, axs = plt.subplots(nrows=2, figsize=(8, 16))

sns.heatmap(df_train.drop(['PassengerId'], axis=1).corr(), ax=axs[0], annot=True, square=True,
            cmap='mako', annot_kws={'size': 14})
sns.heatmap(df_test.drop(['PassengerId'], axis=1).corr(), ax=axs[1], annot=True, square=True,
            cmap='rocket', annot_kws={'size': 14})

for i in range(2):
    axs[i].tick_params(axis='x', labelsize=14)
    axs[i].tick_params(axis='y', labelsize=14)
    
axs[0].set_title('Training correlation set', fontsize=17)
axs[1].set_title('Testing correlation set', fontsize=17)

### Take away from correlation plot
- Following the above plot, there is only few highly correlated features but this signal is a good point. It means that the features don't contain much the redundant or unnecessary values and we're happy that each feature carries with its own unique information. 

## Remark
- This relationship can be used to create new features with "feature transformation" and "feature interaction". 
- Target encoding could be very useful as well because of the high correlations with Survived feature.
- Categorical features have very distinct distributions with different survival rates. Those features can be one-hot encoded. Some of those features may be combined with each other to make new features.

In [ ]:
df_all = concat_df(df_train, df_test)
df_all.head()

<a name='3'></a>
# 3. Feature engineering

<a name='3.1'></a>
## 3.1 Binning Continous features
- Is process of transforming from a continuous or numerical variable to a categorical feature by groupping them into some smaller ranges. We called it "Binning" technique
- Binning of continuous variable is one of feature engineering technique and tends to improve the performance of the model. 
- The binning can be also used to identify missing values or outliers
- This mechanism divived the continuous variable into specific number of bins which is a smaller range out of total range of the numerous feature 

<a name='3.1.1'></a>
### 3.1.1 Fare

In [ ]:
df_all['Fare'] = pd.qcut(df_all['Fare'], 10)

In [ ]:
plt.figure(figsize=(12, 8))
# Countplot for 2 features
sns.countplot(x='Fare', hue='Survived', data=df_all, palette='Accent')
    
plt.xlabel('Fare', size=15, labelpad=15)
plt.ylabel('Passenger Count', size=15, labelpad=15)    
plt.tick_params(axis='x', labelsize=8)
plt.tick_params(axis='y', labelsize=15)
    
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 16})
plt.title('Count of Survival in Fare Feature', size=20, y=1.05)
plt.show()

REMARK
- "Fare" feature and "Survived" rate are negatively correlated with each other
- 13 quantiles based bins are used for 'Fare' feature. They provide a decent amount of information gain
- The group on the left has the lowest survival rate and the group on the right has the highest survival rate
- There is an unusual group of fare's values [21.555, 25.52] which has higher the survival rate

<a name='3.1.2'></a>
### 3.1.2 Age

In [ ]:
df_all['Age'] = pd.qcut(df_all['Age'], 10, duplicates='drop')

In [ ]:
plt.figure(figsize=(15, 8))
# Countplot for 2 features
sns.countplot(x='Age', hue='Survived', data=df_all, palette='GnBu_r')
    
plt.xlabel('Age', size=15, labelpad=15)
plt.ylabel('Passenger Count', size=15, labelpad=15)    
plt.tick_params(axis='x', labelsize=8)
plt.tick_params(axis='y', labelsize=15)
    
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 16})
plt.title('Count of Survival in Age Feature', size=20, y=1.05)
plt.show()

REMARK
- Passenger with ages ranging from [22.0, 25.0] has by far the most lowest survival rate than the other and bin with [0.169, 16] has the highest survival rate
- There is an age's group [34.0, 40.0] with unusual high survival rate in this process

<a name='3.2'></a>
## 3.2 Frequency Encoded Binning

<a name='3.2.1'></a>
### 3.2.1 Family size
- Family_Size is created by adding SibSp, Parch and 1. 
- Those columns are added to compute the total size of family & 1 is added to the end for current passenger himself/herself. The Family_Size table. Graph also clearly show that `Family_size` is the predictor of survival rate, b.c different size turns out different rate
    
    - Family size with 1 labeled: Alone
    - Family Size with 2, 3 and 4 are labeled as Small
    - Family Size with 5, 6, 7, 8 and 11 are labeled as Large
- Because the survival rate is highly effected by whether passenger was alone or not. Therefore, a new feature "Alone" can be considered as a flag for the lonely passengers

In [ ]:
# Family_Size feature
df_all['Family_Size'] = df_all['SibSp'] + df_all['Parch'] + 1
# Family_Size_Grouped feature
family_map = {1:'alone', 2:'small', 3:'small', 4:'small', 5:'large', 6:'large', 7:'large', 8:'large', 11:'large'}
df_all['Family_Size_Grouped'] = df_all['Family_Size'].map(family_map)
# Create feature "Alone"
df_all['Alone'] = 0
df_all['Alone'].loc[df_all['Family_Size_Grouped'] == 'alone'] = 1

In [ ]:
plt.figure(figsize=(10, 7))
sns.countplot(x="Family_Size_Grouped", hue='Survived', data=df_all, palette='Set2')
plt.xlabel('Family_Size_Grouped', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.show()

<a name='3.2.2'></a>
### 3.2.2 Ticket frequency
- Passengers who're comming with their families or their mates bought the same tickets
- There are too many unique `Ticket` values to analyze, so grouping them up by their frequencies makes things easier.

(Ask myself) **How** is this feature difference from `Family_size`?
- Many passengers travel along with groups who **can be their friends, team mates** not necessary be a family. Therefore, they are not counted in `Family_size` but instead using **the same ticket number**

(Ask myself) **How** does the family size affect the survival rate?
- According to graph, passengers traveling with group of 2, 3, 4 have the highest survival rate, while those who travel alone has the lowest survival rate. From group of 5 onwards, the survival rate decreases dramatically. 
- The pattern is really similar to `Family_size` feature --> `Ticket_frequency` can be considered as the predictor

In [ ]:
df_all['Ticket_Frequency'] = df_all.groupby('Ticket')['Ticket'].transform('count')

In [ ]:
plt.figure(figsize=(15, 8))
# Countplot for 2 features
sns.countplot(x='Ticket_Frequency', hue='Survived', data=df_all)
    
plt.xlabel('Ticket_Frequency', size=15, labelpad=15)
plt.ylabel('Survival Count', size=15, labelpad=15)    
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
    
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 16})
plt.title('Count of Survival in Ticket frequency Feature', size=20, y=1.05)
plt.show()

<a name='3.3'></a>
## 3.3 Title & is Married
- Title is created by extracting the prefix before Name feature. According to graph below, there are many titles that are occuring very few times. 
- Some of those titles doesn't seem correct and they need to be replaced. 
- **Miss, Mrs, Ms, Mlle, Lady, Mme, the Countess, Dona** titles are replaced with Miss/Mrs/Ms because all of them are female. 
- **Dr, Col, Major, Jonkheer, Capt, Sir, Don and Rev** titles are replaced by **Dr/Sir/Noble/Clergy** because those passengers have similar characteristics. 
- **Master** is a unique title. It is given to male passengers below age 26. They have the highest survival rate among all males.

**Is_Married** is a binary feature based on the **Mrs** title. Mrs title has the highest survival rate among other female titles. This title needs to be a feature because all female titles are grouped with each other.

In [ ]:
df_all['Name']

In [ ]:
# Extract the tile and Is_married signal from the name
df_all['Title'] = df_all['Name'].str.split(',', expand=True)[1].str.split('.', expand=True)[0].str.strip()
df_all['Married'] = 0
df_all['Married'].loc[df_all['Title'] == 'Mrs'] = 1

# Title unique values count
df_all['Title'].value_counts()

In [ ]:
# Replace Miss, Mrs, Ms, Mlle, Lady, Mme, the Countess, Dona title as "Miss/Mrs/Ms"
df_all['Title'] = df_all['Title'].replace(['Miss', 'Mrs','Ms', 'Mlle', 'Lady', 'Mme', 'the Countess', 'Dona'], 'Miss/Mrs/Ms')
# Replace Dr, Col, Major, Jonkheer, Capt, Sir, Don and Rev title as "Dr/Military/Noble/Clergy"
df_all['Title'] = df_all['Title'].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev', 'Master'], 'Dr/Sir/Military/Master')

df_all['Title'].unique()

<a name='3.4'></a>
## 3.4 Feature Transformation
- After modifying all the features into the appropriate forms, we need to encode the categorical features into numerous so that the data is well-prepared for training model.

In [ ]:
# Train/Test splitting
df_train = df_all.loc[:890]
df_test = df_all.loc[891:]
dfs = [df_train, df_test]

In [ ]:
df_train.head()

<a name='3.4.1'></a>
### 3.4.1 Label encoding non-numerical features
- Norminal data: `Embarked`, `Deck`, `Sex`, `Title`
- Ordinal data: `Family_Size_Grouped`, `Age`, `Fare`

We convert them into numerical data type by using `LabelEncoding`. `LabelEncoding` basically transform categorical value to 0 to n class based on their unique values of each features

Note that:
- **Nominal data** is a type of data that is used as naming or label variables without providing any quantitative value
- **Ordinal data** contains same features as nominal data but plussing the capability of ordered & measured features

In [ ]:
non_numeric_features = ['Embarked', 'Sex', 'Deck', 'Title', 'Family_Size_Grouped', 'Age', 'Fare']

for df in dfs:
    for feature in non_numeric_features:
        df[feature] = preprocessing.LabelEncoder().fit_transform(df[feature])

In [ ]:
df_train[non_numeric_features]

<a name='3.4.2'></a>
### 3.4.2 One-hot Encoding the categorical features
- We do NOT encode `Family_Size_Grouped`, `Age`, `Fare` feature using One-hot encoder b.c they are ordinal data type
- Because the natural ordering between ordinal categories resulting from One-hot encoding may tend to poor performance or unexpected consequence

In [ ]:
cat_features = ['Sex', 'Deck', 'Embarked', 'Title']
# encoded_features: list of encoded df of training & testing dataset
encoded_features = []

# One-hot encoding for both training & testing set
for df in dfs:
    for feature in cat_features:
        # Change to array after encoding b.c want to add columns when change back to df
        encoded_feat = preprocessing.OneHotEncoder().fit_transform(df[feature].values.reshape(-1, 1)).toarray()
        # num of unique value in each feature
        n = df[feature].nunique()
        # "feature_uniqueVal" are the col's names in df after One-hot encoding
        cols = ['{}_{}'.format(feature, n) for n in range(1, n + 1)]
        encoded_df = pd.DataFrame(encoded_feat, columns=cols)
        encoded_df.index = df.index
        encoded_features.append(encoded_df)
        
df_train = pd.concat([df_train, *encoded_features[:4]], axis=1)
df_test = pd.concat([df_test, *encoded_features[4:]], axis=1)

In [ ]:
# For submission dataframe
passengerID = df_test['PassengerId']

drop_cols = ['Deck', 'Embarked', 'Family_Size', 'Name', 'Parch', 
             'PassengerId', 'Sex', 'SibSp', 'Ticket', 'Title']

df_train.drop(columns=drop_cols, inplace=True)
drop_cols.append("Survived")
df_test.drop(columns=drop_cols, inplace=True)

In [ ]:
df_train.head()

## Conclusion
- `Age` and `Fare` features are binned. Binning features helps dealing with outliers. `Family_size` is created by summing value of `Parch` as number of parent's member, `SibSp` as number of children's member and plussing 1 as him/herself. `Ticket_Frequency` is created by counting the frequency of each unique ticket
- `Name` feature is very useful. First, `Title` and `Is_Married` features are created from the title prefix in the names.
- Finally, the non-numeric type features are label encoded and categorical non-ordinal features are one-hot encoded. Created 5 new features (`Family_Size_Grouped`, `Title`, `Is_Married`, `Deck`, `Ticket_Frequency`) and dropped the useless features after encoding.

<a name='4'></a>
# 4. Modeling
Titanic competition is a binary classification and regression problems. We want to determine the relationship between the features (Age, Fare, Sex,...) and the target column (Survived). Based on the given dataset including this type of problem, a category of machine learning which is called Supervised learning should be useful. Therefore we can narrow down our choice to these below supervised algorithms:
- K-Nearest Neightbors (KNN)
- Support Vector Machines
- Naive Bayes Classifier
- Decision tree
- Random forest
- Extra tree
- Logistic regression
- Stochastic gradient Boosting

In [ ]:
X_train = preprocessing.StandardScaler().fit_transform(df_train.drop(['Survived'], axis=1))
y_train = df_train['Survived'].values
X_test = preprocessing.StandardScaler().fit_transform(df_test)

print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))

<a name='4.1'></a>
## 4.1 Cross validate simple models
The goal of any supervised algorithm is to best optimize the mapping function (f) for
the output variable (Y) with given input data (X)
- In order to create an efficient model, training and testing dataset should comes from different distributions so that the model is capable of adapting to the real-world data
- Therefore, in common cases, the performances on training dataset is largely higher than
those on testing dataset. This problem is called **"Variance error"**

**"K-fold Cross validation"** technique provides a smart move to avoid this problem. It split the training data into k equaled groups & perform the prediction process on 1 group, the other groups are used for training the model. The whole process will be repeatedly carried out for each group & averaging the accuracy of them all to get the final result 

In [ ]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction

In [ ]:
# Cross validate models with K-fold cross validation
Kfold = KFold(n_splits=10, random_state=SEED)

In [ ]:
ML_algos = {
    'KNN' : neighbors.KNeighborsClassifier(),
    'SVC' : svm.SVC(probability=True, random_state=SEED),
    'GaussianNB' : naive_bayes.GaussianNB(),
    'DTree' : tree.DecisionTreeClassifier(random_state=SEED),
    'RFor' : ensemble.RandomForestClassifier(random_state=SEED),
    'LReg' : linear_model.LogisticRegressionCV(random_state=SEED),
    'GrB' : ensemble.GradientBoostingClassifier(random_state=SEED),
    'ExTree' : ensemble.ExtraTreesClassifier(random_state=SEED)
}
cv_result = dict()
cv_mean = dict()
cv_std = dict()

# Compute the cross validation result of each ML algorithm
for name, ML_algo in ML_algos.items():
    cv_result[name] = cross_val_score(ML_algo, X_train, y=y_train, scoring='accuracy', cv=Kfold, n_jobs=3)
    cv_mean[name] = cv_result[name].mean()
    cv_std[name] = cv_result[name].std()
    

In [ ]:
ML_cv = pd.DataFrame({'cv_mean' : cv_mean, 'cv_std' : cv_std})
ML_cv

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = sns.barplot(x=ML_cv['cv_mean'], y=ML_cv.index, orient='h')
ax.set_title('ML algorithms cross validation mean values', fontsize=15)
plt.show()

<a name='4.2'></a>
## 4.2 Hyper parameters tuning
- Each model architecture has their own set of parameters which is called "Hyperparameters".
- "Hyperparameters tuning" is a process of constructing clear plan to search for the ideal model architectures by choosing the best options for the their set of parameters 

In [ ]:
# Cleaner & greater ways forhyperparameters
# Hyperparameter Tune with GridSearchCV
n_estimators = [10, 50, 100, 300]

grid_param = [
    # KNN 
    {'n_neighbors' : range(1, 31),
     'weights' : ['uniform', 'distance'],
     'metric' : ['euclidean', 'manhattan', 'minkowski']},
    
    # Support vector machine 
    {'kernel' : ['poly', 'rbf', 'sigmoid'],
     'C' : [50, 10, 1.0, 0.1, 0.01], 
     'gamma' : ['scale']},
    
    # Gradient Boosting 
    {'n_estimators' : n_estimators,
     'learning_rate': [0.001, 0.01, 0.1],
     'subsample' : [0.5, 0.7, 1.0],
     'max_depth' : [3, 7, 9] },
    
    # Random forest 
    {'n_estimators' : n_estimators,
    'max_features' : ['sqrt', 'log2'],
    'criterion' : ['gini', 'entropy'],
    'oob_score' : [True]},
    
    # Logistic regression
    {'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
     'Cs' : [10, 1.0, 0.1, 0.01],
    'fit_intercept': [True, False]}
]

ML_algos = {
    'KNN' : neighbors.KNeighborsClassifier(),
    'SVC' : svm.SVC(probability=True, random_state=SEED),
    'GrB' : ensemble.GradientBoostingClassifier(random_state=SEED),
    'RFor' : ensemble.RandomForestClassifier(random_state=SEED),
    'LReg' : linear_model.LogisticRegressionCV(random_state=SEED)
}

In [ ]:
# Time performance counter
start_time = time.perf_counter()

for param, ML_algo in zip(grid_param, ML_algos.values()):
    # Start counting time
    start = time.perf_counter()
    grid_search = GridSearchCV(estimator = ML_algo, param_grid=param, n_jobs=2, cv=Kfold, 
                               scoring='accuracy',error_score=0)
    grid_result = grid_search.fit(X_train, y_train)
    best_params = grid_result.best_params_
    best_score = grid_result.best_score_
    # Compute time for executing each algo
    run = time.perf_counter() - start
    
    print(f'The best parameter for {ML_algo} is {best_params} with the best accuracy score {best_score}')
    print(f'Computational runtime of this algo: {round(run, 2)} seconds\n')
    ML_algo.set_params(**best_params)

run_total = time.perf_counter() - start_time
print('Total optimization time was {:.2f} minutes.'.format(run_total/60))

<a name='4.3'></a>
## 4.3 Confusion matrix
- Confusion matrix is a table describing the performance measurement of ML classification problems. The technique is performed on the set of validation data for which the true values are known

<img src="https://2.bp.blogspot.com/-EvSXDotTOwc/XMfeOGZ-CVI/AAAAAAAAEiE/oePFfvhfOQM11dgRn9FkPxlegCXbgOF4QCLcBGAs/s1600/confusionMatrxiUpdated.jpg" style="width:450px;height:300px;">

For those who confuse, the "positive" or "negative" depends on the purpose of the model. This model is used to predict the survival passengers --> "positive" means predict 1 as Survived & "negative" means predict 0 as not survived
- **True positive (TP)**: "True" means the prediction value is similar to the actual value AND "positive" means models predict 1 as a survived passenger
- **True negative (TN)**: "True" means the prediction value is similar to the actual value AND "negative" means models predict 0 as a non-survived passenger
- **False negative (FN)**: "False" means the prediction value is different from the actual values AND "negative" means models predict 0 as a non-survived passenger
- **False positive (FN)**: "False" means the prediction value is different from the actual values AND "positive" means models predict 1 as a survived passenger

In [ ]:
for name, model in ML_algos.items():
    y_pred = cross_val_predict(model, X_train, y_train, cv=10)
    
    print(f'Confusion matrix of algorithm {name}')
    print(pd.DataFrame(confusion_matrix(y_train, y_pred),\
                       columns=["Predicted Not-Survived", "Predicted Survived"],\
                       index=["Not-Survived","Survived"]), '\n')
    print(classification_report(y_train, y_pred))
    print('------------------------------\n\n')

<a name='4.4'></a>
## 4.4 Ensemble model
- The voting classifier can be a good choice to combine the predictions coming from the 5 classifiers.

In [ ]:
voteC = ensemble.VotingClassifier(estimators=[('KNN_best', ML_algos['KNN']), ('SVC_best', ML_algos['SVC']),
                                             ('GrB_best', ML_algos['GrB']), ('RFor_best', ML_algos['RFor']),
                                             ('LReg_best', ML_algos['LReg'])],
                                 voting='soft', n_jobs=4)
voteC = voteC.fit(X_train, y_train)

<a name='4.5'></a>
## 4.5 Submission

In [ ]:
# Making the prediction
y_pred = voteC.predict(X_test)

submission = pd.DataFrame({
        'PassengerId' : passengerID,
        'Survived' : y_pred.astype(int)})

submission.to_csv('submission.csv', index=False)

## Credit & Resource
- Credit should be extended for [Anisotropic](https://www.kaggle.com/arthurtok) and [Gunes Evitan](https://www.kaggle.com/gunesevitan) and all super surportive Kaggler. Send a big thanks to all of them!
- To [Dataquest](https://www.dataquest.io/?utm_source=google&utm_medium=cpc&utm_campaign&matchtype=e&device=c&campaignid=2083591861&adgroupid=75850930719&adid=482446438765&gclid=CjwKCAjwxo6IBhBKEiwAXSYBs5UgtShJIv7wWV0xdKvnKNUbQ-YyWCpZoUMlTh4fEpyhjU6ButFI0RoCMRAQAvD_BwE) where I start my journey
- To [Coursera](https://www.coursera.org/) that provided me fundamental skills to get my feets wet
<br>

**Some great resource:**
- For excellent materials on stacking or ensembling model, refers to a must-read article for all Kagglers [Kaggle Ensembling Guide](https://mlwave.com/kaggle-ensembling-guide/)
- [Introduction to machine learning with Python: A Guide to data scientist](https://www.amazon.com/gp/product/1449369413/ref=as_li_tl?ie=UTF8&tag=kaggle-20&camp=1789&creative=9325&linkCode=as2&creativeASIN=1449369413&linkId=740510c3199892cca1632fe738fb8d08) written by core creators of sklearn
- [Machine learning online course](https://www.coursera.org/learn/machine-learning?) hosted by Andrew Ng _ a pioneer in online education especially on ML & AI

### I tried my best to finish this notebook with all my current abilities. I honestly hope somehow Kagglers found this notebook helpful. Sharing is caring!